### Load MetaMap instance

In [55]:
from pymetamap import MetaMap

file_path = "/Users/soulofshadow/Desktop/UMLSextract/public_mm/bin/metamap18"
mm = MetaMap.get_instance(file_path)

In [56]:
#starting Metamap server
'''
cd /Users/soulofshadow/Desktop/UMLSextract/public_mm

The SKR/Medpost Part-of-Speech Tagger Server
./bin/skrmedpostctl start

Word Sense Disambiguation (WSD) Server (optional)
./bin/wsdserverctl start
'''

mm.get_instance

<function pymetamap.MetaMap.MetaMap.get_instance(metamap_filename, version=None, backend='subprocess', **extra_args)>

In [57]:
#test server actiavted
sents = ['Heart Attack', 'John had a huge heart attack']

concepts,error = mm.extract_concepts(sents)

### Load dataset

In [85]:
import os
import numpy as np
import pandas as pd

dataset_path = "/Users/soulofshadow/Desktop/UMLSextract/sentences.csv"

In [ ]:
#get the text dataset
# sentences = pd.read_csv(dataset_path, header=None, sep="\t")
sentences = pd.read_csv(dataset_path)
# sentences.columns = ['idk','text']
sentences = sentences.drop(['Unnamed: 0'], axis = 1)

In [87]:
print(len(sentences))
sentences.head()

427765


,text,entity
0,The hepatic ultrastructural aspect and the hep...,"[{'cui': 'C0041623', 'name': 'Ultrastructure',..."
1,The group G streptococcus may be a more common...,"[{'cui': 'C0014118', 'name': 'Endocarditis', '..."
2,Effects of a 24 hour fast were studied in 21 o...,"[{'cui': 'C0028754', 'name': 'Obesity', 'type'..."
3,The present study has been undertaken to elici...,"[{'cui': 'C0342895', 'name': 'Fish-Eye Disease..."
4,Selected androgenic and nonandrogenic steroids...,"[{'cui': 'C0002844', 'name': 'Androgens', 'typ..."


In [95]:
sentences['entity'] = sentences['entity'].where(sentences['entity'].notnull(), None)

### Extract entities

##### version 1 with iterrows()

In [ ]:
from tqdm import tqdm
import re
reObj = re.compile("\D")

for index, item in sentences.iterrows():
    concepts, error = mm.extract_concepts([item['text']])

    if item['entity'] is not None:
        continue;

    entities = []
    for concept in concepts:
        if concept.__class__.__name__ == 'ConceptMMI':
            entity = {}
            entity['cui'] = concept[4]
            entity['name'] = concept[3]
            entity['type'] = concept[5]

            position = reObj.split(concept[8])
            position = [int(i) for i in position if i != '']
            position = position[:2]

            entity['pos'] = position
            entity['triger'] = item['text'][position[0]: position[0] + position[1]]

            entities.append(entity)
    
    item['entity'] = entities

In [91]:
sentences.to_csv('sentences.csv')

In [88]:
for index, item in sentences.iterrows():
    if np.isnan(item['entity']):
        print(index)
        break

##### version 2 with tqdm

In [97]:
from tqdm import tqdm
tqdm.pandas()
pd.options.mode.chained_assignment = None  # default='warn'

import re
reObj = re.compile("\D")

In [98]:
def find_entity(sentence, entities):
    if entities is not None:
        return entities
    
    concepts, error = mm.extract_concepts([sentence])
    entities = []
    for concept in concepts:
        if concept.__class__.__name__ == 'ConceptMMI':
            entity = {}
            entity['cui'] = concept[4]
            entity['name'] = concept[3]
            entity['type'] = concept[5]

            position = reObj.split(concept[8])
            position = [int(i) for i in position if i != '']
            position = position[:2]

            entity['pos'] = position
            entity['triger'] = sentence[position[0]: position[0] + position[1]]

            entities.append(entity)
    return entities

In [99]:
sentences['entity'] = sentences.progress_apply(lambda x: find_entity(x['text'], x['entity']), axis = 1)

  1%|          | 4563/427765 [00:00<00:16, 25734.50it/s]### UNKNOWN lexcat for [baw,baw,baw,washing,washing,'wash (cleansing action)']|ev(-719,'C0441648','Washing','Wash (cleansing action)',[washing],[acty],[[[2,2],[1,1],1]],[2],2,yes,no,[],[1339/3],0,0)|['.'-adj,after-prep,both-det,bronchoalveolar-noun,'bronchoalveolar washings'-noun,i-adj,'i.n'-adj,n-adj,washings-noun]
### UNKNOWN lexcat for [baw,baw,baw,washing,washing,'wash (cleansing action)']|ev(-627,'C0441648','Washing','Wash (cleansing action)',[washing],[acty],[[[2,2],[1,1],1]],[2],2,no,no,[],[1403/3],0,0)|[antibody-noun,bronchoalveolar-noun,'bronchoalveolar washings'-noun,washings-noun]
### UNKNOWN lexcat for [baw,baw,baw,washing,washing,'wash (cleansing action)']|ev(-560,'C0441648','Washing','Wash (cleansing action)',[washing],[acty],[[[5,5],[1,1],1]],[5],5,no,no,[],[1442/3],0,0)|[bronchoalveolar-noun,'bronchoalveolar washings'-noun,in-prep,ratio-noun,the-det,washings-noun]
  1%|▏         | 6391/427765 [00:15<22:36, 310.60it